# Machine learning model using random forest regressor
The aim of this notebook is to create a machine learning model that can predict the attendance time. The benefit of having such model, is that when certain parameters, like weather or borough are provided the fire department could give good estimates on the arrival time.

When modeling, all the "incidents data" that was cleaned up and saved previously, is used.

To be able to fit the data to the model one hot encoding was done on the categorical values. To be able to render the bokeh plot on the website the parameters had to be limited. The snow depth, property category and incident group type were used.

In [3]:
# Import necessary libraries and apply settings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from IPython.display import Image
%matplotlib inline
import json
import fiona
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
from bokeh.palettes import Reds6 as palette
from bokeh.plotting import figure, save
from bokeh.resources import CDN
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, shape
from shapely.prepared import prep
import geopandas as gpd
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, LogColorMapper
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

incidents = pd.read_csv('fire_incidents2020.csv')#pd.read_csv('fire_incidents1year.csv')
incidents = incidents.append(pd.read_csv('fire_incidents2019.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2018.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2017.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2016.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2015.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2014.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2013.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2012.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2011.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2010.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2009.csv'));

In [4]:
incidents = incidents.dropna(axis=0, subset=['AttendanceTimeSeconds'])
incidents = incidents[['IncidentNumber', 'DateOfCall', 'CalYear', 'TimeOfCall',
       'HourOfCall', 'IncidentGroup', 'StopCodeDescription',
       'SpecialServiceType', 'PropertyCategory', 'PropertyType',
       'AddressQualifier', 'Postcode', 'Postcode_district',
       'IncGeo_BoroughCode', 'ProperCase', 'WardCode',
       'Easting', 'Northing', 'FRS', 'IncidentStationGround',
       'FirstPumpArriving_AttendanceTime',
       'FirstPumpArriving_DeployedFromStation',
       'SecondPumpArriving_AttendanceTime',
       'SecondPumpArriving_DeployedFromStation',
       'NumStationsWithPumpsAttending', 'NumPumpsAttending', 'PumpCount',
       'PumpHoursRoundUp', 'Notional Cost (£)', 
        'Population', 'Households','Altitude',
       'London zone', 'Average Income', 'Latitude', 'Longitude', 'WardName',
       'ResourceMobilisationID', 'Resource_Code', 'AttendanceTimeSeconds',
        'DeployedFromStation_Code',
       'DeployedFromStation_Name', 'DeployedFromLocation', 'MobilisationOrder',
       'PlusCode_code', 'PlusCode_Description', 'DelayCodeID',
       'DelayCode_Description', 'PerformanceReporting']]

# Convert datetime to date
incidents.DateOfCall = pd.to_datetime(incidents.DateOfCall)

# Add columns for month, week, weekday
incidents['Month'] = incidents.DateOfCall.dt.month
incidents['Week'] = incidents.DateOfCall.dt.week
incidents['Weekday'] = incidents.DateOfCall.dt.weekday # 0 is monday, 6 is sunday

# convert dateofcall to date not datetime for later join
incidents.DateOfCall = pd.to_datetime(incidents.DateOfCall).dt.date

# use data until 2017
enddate = pd.to_datetime("2017-12-31").date()
incidents = incidents[incidents.DateOfCall <= enddate]

incidents.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,DeployedFromLocation,MobilisationOrder,PlusCode_code,PlusCode_Description,DelayCodeID,DelayCode_Description,PerformanceReporting,Month,Week,Weekday
18,013558-01022017,2017-02-01,2017,14:30:10,14,False Alarm,False alarm - Good intent,NaN,Non Residential,Single shop,...,NaN,1.0,Initial,Initial Mobilisation,8.0,Traffic calming measures,1st,2,5,2
29,020475-18022017,2017-02-18,2017,10:26:29,10,Special Service,Special Service,Effecting entry/exit,Non Residential,Purpose built office,...,Home Station,1.0,Initial,Initial Mobilisation,NaN,NaN,1st,2,7,5
37,027279-05032017,2017-03-05,2017,18:50:45,18,False Alarm,AFA,NaN,Non Residential,Purpose built office,...,Home Station,1.0,Initial,Initial Mobilisation,12.0,Not held up,1st,3,9,6
45,030099-12032017,2017-03-12,2017,17:38:17,17,Special Service,Special Service,Effecting entry/exit,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,Home Station,1.0,Initial,Initial Mobilisation,NaN,NaN,1st,3,10,6
56,033273-19032017,2017-03-19,2017,17:00:00,17,Special Service,Special Service,Effecting entry/exit,Outdoor Structure,Railings,...,Home Station,1.0,Initial,Initial Mobilisation,NaN,NaN,1st,3,11,6


## Adding the weather data and merging it to the incident records.

There was a problem with reading in the processed csv file so the reading and merging is done again here.

In [5]:
# Create first dataframe and define structure
weather = pd.read_csv('WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2009.csv', 
                    parse_dates = True,
                    usecols = ['ob_time', 'wind_speed_unit_id','wind_direction', 'wind_speed', 'prst_wx_id', 'visibility', 'msl_pressure', 'air_temperature', 'snow_depth', 'drv_hr_sun_dur'],
                    header=280)
weather = weather[:-1] # skip the last line
weather.ob_time = pd.to_datetime(weather.ob_time)

files = ['WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2010.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2011.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2012.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2013.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2014.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2015.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2016.csv',
        'WeatherData/midas-open_uk-hourly-weather-obs_dv-201901_greater-london_00708_heathrow_qcv-1_2017.csv']

# Append each additional file onto the datafile
for f in files:
    w_df = pd.read_csv(f, 
                    parse_dates = True,
                    usecols = ['ob_time', 'wind_speed_unit_id','wind_direction', 'wind_speed', 'prst_wx_id', 'visibility', 'msl_pressure', 'air_temperature', 'snow_depth', 'drv_hr_sun_dur'],
                    header=280)
    w_df = w_df[:-1] # Skip the last line
    w_df.ob_time = pd.to_datetime(w_df.ob_time)
    weather = weather.append(w_df, ignore_index = True)
weather.tail()

,ob_time,wind_speed_unit_id,wind_direction,wind_speed,prst_wx_id,visibility,msl_pressure,air_temperature,snow_depth,drv_hr_sun_dur
78826,2017-12-31 19:00:00,4.0,210.0,16.0,NaN,1600.0,992.3,8.5,NaN,0.0
78827,2017-12-31 20:00:00,4.0,210.0,14.0,NaN,1200.0,992.1,8.5,NaN,0.0
78828,2017-12-31 21:00:00,4.0,230.0,20.0,23.0,4000.0,992.1,8.2,NaN,0.0
78829,2017-12-31 22:00:00,4.0,260.0,25.0,81.0,2900.0,993.7,7.5,NaN,0.0
78830,2017-12-31 23:00:00,4.0,260.0,20.0,23.0,4500.0,995.7,7.2,NaN,0.0


In [6]:
weather_codes = pd.DataFrame(columns=["Code", "Description"],
                            data=[[0, 'No precipitation, fog, ice fog, duststorm, sandstorm, drifting or blowing snow at the station at the time of observation'],
                                 [10, 'No precipitation, fog, ice fog, duststorm, sandstorm, drifting or blowing snow at the station at the time of observation'],
                                 [20, 'Precipitation, fog, ice fog or thunderstorm at the station during the preceding hour but not at the time of observation'],
                                 [30, 'Duststorm, sandstorm, drifting or blowing snow'],
                                 [40, 'Fog or ice fog at the time of observation'], 
                                 [50, 'Drizzle'], 
                                 [60, 'Rain'], 
                                 [70, 'Solid precipitation not in showers'], 
                                 [80, 'Showery precipitation, or precipitation with current or recent thunderstorm'], 
                                 [90, 'Showery precipitation, or precipitation with current or recent thunderstorm'], 
                                 ])
weather_codes

weather['Date'] = weather.ob_time.dt.date
weather['Hour'] = weather.ob_time.dt.hour
weather['prst_wx_id_rounded'] = weather.prst_wx_id.round(decimals=-1)
weather_merged = pd.merge(weather, weather_codes, how='left', left_on='prst_wx_id_rounded', right_on='Code')

In [7]:
df = pd.merge(incidents, weather_merged, how='left', left_on=['DateOfCall', 'HourOfCall'], right_on=['Date', 'Hour'])

# Remove columns
del df['ob_time']
del df['Date']
del df['Hour']
del df['prst_wx_id']
del df['prst_wx_id_rounded']
del df['Code']

df.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,Weekday,wind_speed_unit_id,wind_direction,wind_speed,visibility,msl_pressure,air_temperature,snow_depth,drv_hr_sun_dur,Description
0,013558-01022017,2017-02-01,2017,14:30:10,14,False Alarm,False alarm - Good intent,NaN,Non Residential,Single shop,...,2,4.0,180.0,8.0,3000.0,1005.6,10.9,NaN,0.1,NaN
1,020475-18022017,2017-02-18,2017,10:26:29,10,Special Service,Special Service,Effecting entry/exit,Non Residential,Purpose built office,...,5,4.0,190.0,7.0,430.0,1026.1,7.0,NaN,0.0,"No precipitation, fog, ice fog, duststorm, san..."
2,027279-05032017,2017-03-05,2017,18:50:45,18,False Alarm,AFA,NaN,Non Residential,Purpose built office,...,6,4.0,230.0,15.0,900.0,991.6,6.7,NaN,0.0,"Showery precipitation, or precipitation with c..."
3,030099-12032017,2017-03-12,2017,17:38:17,17,Special Service,Special Service,Effecting entry/exit,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,6,4.0,290.0,3.0,800.0,1016.0,12.2,NaN,0.0,"No precipitation, fog, ice fog, duststorm, san..."
4,033273-19032017,2017-03-19,2017,17:00:00,17,Special Service,Special Service,Effecting entry/exit,Outdoor Structure,Railings,...,6,4.0,250.0,19.0,2800.0,1007.6,13.9,NaN,0.1,NaN


## Data preperation for model

Preparing the data in a way so the model can understand it.

In [8]:
df_selected = df[['IncidentGroup','PropertyCategory','IncGeo_BoroughCode',
                  'AttendanceTimeSeconds','snow_depth']].copy()



# remove bourough_codes E00000000
df_selected = df_selected[df_selected.IncGeo_BoroughCode != 'E00000000']

bourough_codes = list(df_selected.IncGeo_BoroughCode.unique())
Incident_groups = np.sort(list(df_selected.IncidentGroup.unique()))
column_names = list(df_selected.columns)
property_types = np.sort(list(df_selected.PropertyCategory.unique()))

To keep as much data as possible the nan values of snow depth are replaced with 0 since nan values would represent no snow.

In [9]:
# Change value to 0 if nan in snow_depth, and drop nan values in attendance
df_selected["snow_depth"] = df_selected["snow_depth"].fillna(0)

# THe model cant take in nan value svo those are dropped
df_selected.dropna(inplace=True)
df_selected.isnull().sum()
df_selected.head()

,IncidentGroup,PropertyCategory,IncGeo_BoroughCode,AttendanceTimeSeconds,snow_depth
0,False Alarm,Non Residential,E09000021,591.0,0.0
1,Special Service,Non Residential,E09000001,232.0,0.0
2,False Alarm,Non Residential,E09000013,361.0,0.0
3,Special Service,Dwelling,E09000012,226.0,0.0
4,Special Service,Outdoor Structure,E09000012,300.0,0.0


Since there are categorical values, those have to be change by one hot encoding.

In [10]:
# Create dummy variables for Borough and PropertyCategory
df_selected = pd.get_dummies(df_selected, prefix="", prefix_sep='',columns=["IncGeo_BoroughCode"])
df_selected = pd.get_dummies(df_selected, prefix="", prefix_sep='',columns=["PropertyCategory"])
df_selected = pd.get_dummies(df_selected, prefix="", prefix_sep='',columns=["IncidentGroup"])
df_selected.head()

,AttendanceTimeSeconds,snow_depth,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,...,Dwelling,Non Residential,Other Residential,Outdoor,Outdoor Structure,Rail Vehicle,Road Vehicle,False Alarm,Fire,Special Service
0,591.0,0.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,232.0,0.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,361.0,0.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,226.0,0.0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,300.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


## Setting up the random forest regressor

There are many parameters that can be tuned in sklearn's random forest regressor. A simple grid search of the values for n_estimators, max_depth and min_samples_split

In [25]:
# Import the modules
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import ensemble

In [27]:
#Split the dataset into features X and target variable Y.
X = df_selected.copy()
del X['AttendanceTimeSeconds']
y = df_selected.AttendanceTimeSeconds.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 70% training and 30% test


In [30]:
# defining the grid from where we want to find the optimal parameters for 

# Define the number of trees used
n_estimators = [100, 200]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 15, num = 4)]
max_depth.append(None)

# Minimum number of samples to split node
min_samples_split = [2, 5, 10]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

In [32]:
regr = RandomForestRegressor()
regr_random = RandomizedSearchCV(estimator = regr, param_distributions = random_grid,
                                 cv = 2, verbose=3, random_state=42, n_jobs = -1);
# Fit the random search model
regr_random.fit(X_train, y_train);

Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed: 12.4min remaining: 70.4min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 15.3min remaining: 15.3min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 17.0min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 18.4min finished


In [33]:
# best combination of parameters
regr_random.best_params_

{'n_estimators': 100, 'min_samples_split': 10, 'max_depth': None}

Now lets used our besta parameters based on the small grid search performed with the cross validation of 2.

In [36]:
regr = RandomForestRegressor(n_estimators = 100, max_depth=None, min_samples_split=10 ,random_state=0)

# Train the model on training data
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

Now the model is used to predict on the training set and see how the performance is by evaluating the R2 score.

In [37]:
# see how predicts on the test instance
y_pred = regr.predict(X_test)

To be able to evaluate the performance of the model 2 functions where made both for classification and regression.

In [59]:
# Import functions to use for model performance
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

# function to evaluate predictions of classification
def evaluate(y_true, y_pred):
    # calculate and display confusion matrix
    labels = np.unique(y_true)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print('Confusion matrix\n- x-axis is true labels \n- y-axis is predicted labels')
    print(cm)
    # calculate precision, recall, and F1 score
    accuracy = float(np.trace(cm)) / np.sum(cm)
    precision = precision_score(y_true, y_pred, average=None, labels=labels)[1]
    recall = recall_score(y_true, y_pred, average=None, labels=labels)[1]
    f1 = 2 * precision * recall / (precision + recall)
    print("accuracy:", accuracy)
    print("precision:", precision)
    print("recall:", recall)
    print("f1 score:", f1)
    
# function to evaluate predictions of cintinous variable
def evaluate_continues(y_true, y_pred,regresson):
    # Manual calculation of R2, as (1 - u/v)
    u = ((y_true - y_pred) ** 2).sum() # u
    v = ((y_true - y_true.mean()) ** 2).sum() # 
    R2 = (1 - (u/v))
    gap = abs(np.asarray(y_pred) -  np.asarray(y_true)).sum() / np.asarray(y_true).sum()
    # with function
    print("The R2 score is: ",regresson.score(X_test,y_test))
#     print("The R2 score is: ",R2)
    print("The % gap is: ",gap)

Evaluating the performance is important to see if the model is performing well or not. It is also important to check if the model is overfitting to the training set, this would lead to bad predictions on new incoming data if it would not resamble the training set well enough.

In [60]:
evaluate_continues(y_test,y_pred,regr)

The R2 score is:  0.07628515017186976
The % gap is:  0.332358337815951


In [56]:
# Check to see if we are overfitting
# Check how the model performs on the training set and compare to the results from the test data
y_train_pred = regr.predict(X_train)
evaluate_continues(y_train, y_train_pred,regr)

The R2 score manual is:  0.08401353609186002
The % gap is:  0.3038626479972699


The model is performing okay and the R2 score is at least not below 0, so predicting only the mean value would not give a better results. The model is not overfitting since the R2 values are pretty close. The grid search of parameters values and the cross validation was considered enough to showcase the knowledge. There are many additions available for the model but the time was rather spent on creating visualisation for the model´s results. To do this a charopleth graph is created and the attendance time is predicted based on given value that the model relies on.

To be able to run through all instances a template for the attributes is created, and to be able to select different value all instances have to be run based on the regressor. This is then saved to a dataframe.

In [47]:
template = df_selected.iloc[0].copy()
template[:] = 0
print(template)
template = list(template)
template = np.delete(template, 1) # remove time attendance as we are predicting that

AttendanceTimeSeconds    0.0
snow_depth               0.0
E09000001                0.0
E09000002                0.0
E09000003                0.0
E09000004                0.0
E09000005                0.0
E09000006                0.0
E09000007                0.0
E09000008                0.0
E09000009                0.0
E09000010                0.0
E09000011                0.0
E09000012                0.0
E09000013                0.0
E09000014                0.0
E09000015                0.0
E09000016                0.0
E09000017                0.0
E09000018                0.0
E09000019                0.0
E09000020                0.0
E09000021                0.0
E09000022                0.0
E09000023                0.0
E09000024                0.0
E09000025                0.0
E09000026                0.0
E09000027                0.0
E09000028                0.0
E09000029                0.0
E09000030                0.0
E09000031                0.0
E09000032                0.0
E09000033     

Iterating through all the desired instances and inserting into new dataframe

In [48]:
new_df = pd.DataFrame(columns=column_names) # create new dataframe for results

# predict on specific values for all borough codes
for (index,boroughC) in enumerate(bourough_codes,start=1):
    for (index2,incidentG) in enumerate(Incident_groups,start=43):
        for (index3,propT) in enumerate(property_types,start=34):
            for snowD in np.arange(0,5,2):
                template = np.zeros(46)
                template[index] = 1 # set borough code
                template[0] = snowD # set snow depth
                template[index2] = 1 # set incident group
                template[index3] = 1 # set property type
                predict_set = np.reshape(template, (1, -1)) # make numpy array on right form
                y_pred = regr.predict(predict_set) # predict based on values
                template[index] = 0 # revert borough
                template[index2] = 0 # revert incident group
                template[index3] = 0 # revert property type
                output = [incidentG,propT,boroughC,y_pred[0],snowD]
                data_to_append = {}
                for i in range(len(new_df.columns)):
                    data_to_append[new_df.columns[i]] = output[i]
                
                new_df = new_df.append(data_to_append, ignore_index = True)


In [49]:
new_df.sort_values('AttendanceTimeSeconds')

,IncidentGroup,PropertyCategory,IncGeo_BoroughCode,AttendanceTimeSeconds,snow_depth
389,Special Service,Non Residential,E09000019,130.923730,4
388,Special Service,Non Residential,E09000019,132.277063,2
540,Special Service,Aircraft,E09000025,142.422380,0
541,Special Service,Aircraft,E09000025,148.560957,2
542,Special Service,Aircraft,E09000025,169.507504,4
...,...,...,...,...,...
1676,Special Service,Aircraft,E09000015,764.925575,4
2348,Special Service,Road Vehicle,E09000032,775.285248,4
2347,Special Service,Road Vehicle,E09000032,792.106604,2
2324,Special Service,Aircraft,E09000032,836.809446,4


In [50]:
import pickle
with open('new_df.pkl', 'wb') as f:
    pickle.dump(new_df, f)

In [51]:
unpickled_df = pd.read_pickle("new_df.pkl")

In [52]:
len(unpickled_df)

2673

In [53]:
unpickled_df.head()

,IncidentGroup,PropertyCategory,IncGeo_BoroughCode,AttendanceTimeSeconds,snow_depth
0,False Alarm,Aircraft,E09000021,268.745157,0
1,False Alarm,Aircraft,E09000021,275.411820,2
2,False Alarm,Aircraft,E09000021,272.290471,4
3,False Alarm,Boat,E09000021,221.635755,0
4,False Alarm,Boat,E09000021,234.078476,2


### Now the dataframe is ready for the visulisation